In [2]:
# from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI


# llm = OpenAI(model_name="gpt-3.5-turbo-1106") 
chat = ChatOpenAI(
    temperature=0.1
)

In [10]:
# 3.1 predict message
from langchain.schema import HumanMessage, AIMessage, SystemMessage


prompt = [
    SystemMessage(content="너는 지리학 전문가야, 오로지 한국어로만 대답할 수 있어"),
    AIMessage(content="안녕 나는 구르미야"),
    HumanMessage(content="한국과 일본 사이의 거리는 얼마인가요? 그리고 너의 이름은 뭔가요?")
]

chat.predict_messages(messages=prompt)

AIMessage(content='안녕하세요, 제 이름은 구르미입니다. 한국과 일본 사이의 거리는 직선거리로 약 900km 정도입니다. 그러나 실제로 이동하는 경우에는 해상이나 공기로 이동하므로 더 길어질 수 있습니다.')

In [5]:
# 3.2 prompt template
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate
# template = PromptTemplate.from_template("{country_a}과 {country_b} 사이의 거리는 얼마인가요? 그리고 너의 이름은 뭔가요?")
# prompt = template.format(country_a="멕시코", country_b="태국")

template = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 지리학 전문가야, 오로지 {language}로만 대답할 수 있어"),
        ("ai", "안녕 나는 {name}야"),
        ("human", "{country_a}과 {country_b} 사이의 거리는 얼마인가요? 그리고 너의 이름은 뭔가요?"),
    ]
)
prompt = template.format_messages(
    language="한국어",
    name="륜륜이",
    country_a="중국",
    country_b="미국"
)

chat.predict_messages(prompt)

AIMessage(content='중국과 미국 사이의 거리는 대략 11,000km 정도입니다. 내 이름은 륜륜이에요.')